In [1]:
#Start with pip install -r requirements.txt
import pandas as pd
from functools import partial
from itertools import permutations
import pickle
from random import sample
from statistics import stdev, mean

pd.set_option("precision", 3)

# Hong & Page - replication runs

The models in Hong & Page is run 50 times, with three different parameter combinations presented. Here the fourth permutation (large teams with many heuristics) is included, and the number of runs increased to 500. This takes several hours on a laptop - so I preferred to do it on Google Cloud Engine, using [pyscript2gce](https://github.com/LukasWallrich/pyscript2gce-production/releases/tag/HP-ABM-replication), where a 16-core VM takes less than an hour. See `run_simulation.py` for the code run.

In [5]:
with open("HPmodel_results.pkl",'rb') as f:
    res = pickle.load(f)

In [6]:
# Function to add prefix to columns that describe teams (not individual agents)
def renamer(col, prefix):
    if col.endswith("agent"):
        return col
    else:
        return prefix + col

# Unnest solution dictionary into two columns
res = pd.concat([res.drop(["solution"], axis = 1), res.solution.apply(pd.Series).add_suffix("_solution")], axis = 1)

# Unnest nested dictionary describing the teams 
res_random = res.agent_descriptives.apply(pd.Series).random.apply(pd.Series).rename(mapper = partial(renamer, prefix = "random_"), axis = "columns")
res_best = res.agent_descriptives.apply(pd.Series).best.apply(pd.Series).rename(mapper = partial(renamer, prefix = "best_"), axis = "columns")
res = pd.concat([res.drop(["agent_descriptives"], axis=1), res_best, res_random[res_random.columns[pd.Series(res_random.columns).str.startswith('random_')]]], axis=1)

# Add id variable and rename ambiguous column
res["run_id"] = res.reset_index().index
res = res.rename(columns={"best_agent": "top_agent"})

In [9]:
# Pivot so that random and best groups can be easily compared
# Function to identify columns to be pivoted
def check_var(col_name):
    return not(col_name.find("random_") != -1 or col_name.find("best_") != -1)

col_names = res.columns.values.tolist()

id_cols = list(filter(check_var, col_names))

# Pivot longer, then wider
out = pd.melt(res, id_cols)
out = out.join(out.variable.str.split("_", expand = True)).rename(columns={0:"team_type"}).pivot_table(index=id_cols + ["team_type"], columns=[1], values="value").reset_index()

out["NPdiversity"] = out["NPdiversity"] * 100 # Convert to percentages


In [10]:
# Summarise performance and diversity of best versus random teams
tbl = (out[["team_type", "N_agents", "l", "solution", "NPdiversity"]]
           .groupby(["N_agents", "l", "team_type"])
           .describe().loc[:, (slice(None), ["mean", "std"])])

tbl = tbl.round(2)

sol = (pd.DataFrame(tbl[('solution', 'mean')]
           .astype(str) + " (" +  tbl[('solution', 'std')].astype(str) + ")"))
sol.columns = ["Solution"]

div = (pd.DataFrame(tbl[('NPdiversity', 'mean')]
           .astype(str) + " (" +  tbl[('NPdiversity', 'std')].astype(str) + ")"))
div.columns = ["Diversity"]

sol = sol.join(div)

sol.to_latex("Table1.tex", caption="Results of 500 runs of Hong & Page model")

sol


Solution     Diversity
N_agents l  team_type                            
10       12 best       92.34 (1.26)  84.63 (4.24)
            random     94.35 (0.56)  91.75 (2.52)
         20 best       93.55 (1.25)  87.06 (4.54)
            random     95.73 (0.48)  95.06 (1.77)
20       12 best       93.66 (0.82)  85.79 (3.01)
            random     94.74 (0.47)  91.82 (1.12)
         20 best        95.0 (0.83)  88.52 (3.36)
            random     96.48 (0.42)  95.05 (0.91)

The share of landscapes won by best vs random teams could be interesting - but here, the random teams essentially always win.

In [11]:
# Calculate shares of runs where random_solution > best_solution etc
res["random_winner"] = res["random_solution"] > res["best_solution"]
res["random_winner"] = res["random_winner"].astype(int) * 100

res["best_winner"] = res["random_solution"] < res["best_solution"]
res["best_winner"] = res["best_winner"].astype(int) *100

res["tie"] = res["random_solution"] == res["best_solution"]
res["tie"] = res["tie"].astype(int) * 100

win_rates = (res[["random_winner", "tie", "best_winner", "N_agents", "l"]]
                .groupby(["N_agents", "l"]).describe().loc[:, (slice(None), ["mean"])])

win_rates.columns = win_rates.columns.get_level_values(0)

win_rates["odds"] = win_rates["random_winner"]/win_rates["best_winner"]

win_rates


random_winner  tie  best_winner     odds
N_agents l                                           
10       12           98.4  0.0          1.6   61.500
         20           98.6  0.0          1.4   70.429
20       12           96.8  0.0          3.2   30.250
         20           99.8  0.0          0.2  499.000

In [12]:
#Describe agents per team
(res[["worst_agent", "top_agent", "best_team_average", "random_team_average", "N_agents", "l"]]
    .groupby(["N_agents", "l"]).describe().loc[:, (slice(None), ["mean", "std"])])

worst_agent        top_agent        best_team_average         \
                   mean    std      mean    std              mean    std   
N_agents l                                                                 
10       12      83.638  0.560    86.230  0.550            86.088  0.540   
         20      83.477  0.565    86.458  0.536            86.321  0.530   
20       12      83.680  0.614    86.260  0.571            86.039  0.562   
         20      83.507  0.593    86.483  0.561            86.286  0.551   

            random_team_average         
                           mean    std  
N_agents l                              
10       12              85.026  0.550  
         20              85.063  0.543  
20       12              85.064  0.570  
         20              85.101  0.558

# Divergences from results by Hong and Page

## Standard deviations

Hong & Page state in their article that they report standard deviations of their results. Personal communication later clarified that they in fact reported standard errors. Prior to that, the below analyses showed that their figures could not be standard deviations.

In [10]:
# z-score based on H&P, if they actually reported a standard deviation
print("z-score would be: " + str(round((93.2-92.56)/.02,2)))

# Simulate SD of random teams across 10,000 draws
heuristics = list(permutations(range(1, 13), 3))

res = []
for i in range(10000):
    team = sample(heuristics, 10)
    pairs = permutations(team, 2)
    res.append(mean((len(x[0])-sum(z == y for z, y in zip(x[0], x[1])))/len(x[0]) for x in pairs))
print("SD Teams of 10: " + str(round(stdev(res),3)))

res = []
for i in range(10000):
    team = sample(heuristics, 20)
    pairs = permutations(team, 2)
    res.append(mean((len(x[0])-sum(z == y for z, y in zip(x[0], x[1])))/len(x[0]) for x in pairs))
print("SD Teams of 20: " + str(round(stdev(res),3)))

z-score would be: 32.0
SD Teams of 10: 0.024
SD Teams of 20: 0.011


## Diversity in group of best agents

Grim et al. report heuristics of best-performing agents on 10 fully random landscapes in Table 1. These groups have a mean diversity of 83% (range: 76% to 89%) as calculated below.

In [13]:
# Assess HP diversity
def assess_hp_diversity(heuristic1, heuristic2):
    res = (len(heuristic1)-sum(x == y for x, y in zip(heuristic1, heuristic2)))/len(heuristic1)
    return res

# Best teams and their heuristics taken from Grim et al. (2019)
best_teams = [[(12, 4, 5), (12, 2, 4), (12, 5, 4), (12, 4, 2), (5, 12, 4), (4, 12, 2), (6, 12, 4), (4, 5, 12), (12, 4, 6)], 
[(5, 7, 6), (10, 8, 7), (8, 7, 10), (7, 10, 8), (7, 5, 6), (7, 8, 10), (11, 10, 8), (5, 6, 7), (10, 11, 8)], 
[(1, 10, 3), (1, 6, 2), (1, 3, 10), (3, 1, 10), (6, 2, 1), (10, 3, 1), (10, 1, 3), (1, 10, 6), (7, 5, 3)], 
[(11, 4, 1), (12, 2, 8), (11, 2, 12), (4, 11, 1), (11, 1, 4), (4, 1, 11), (12, 11, 2), (5, 8, 2), (8, 12, 2)], 
[(6, 1, 2), (3, 6, 1), (6, 1, 3), (1, 2, 7), (3, 6, 2), (1, 3, 6), (2, 6, 7), (7, 1, 2), (1, 2, 6)], 
[(4, 8, 7), (3, 4, 8), (4, 8, 3), (7, 4, 8), (4, 3, 8), (1, 8, 7), (3, 8, 4), (3, 8, 7), (8, 7, 2)], 
[(3, 12, 1), (1, 3, 12), (12, 1, 3), (3, 1, 12), (8, 3, 12), (11, 12, 8), (1, 8, 12), (12, 1, 8), (12, 3, 1)], 
[(2, 6, 11), (11, 2, 6), (6, 11, 2), (11, 6, 2), (6, 2, 11), (9, 6, 11), (2, 11, 6), (11, 9, 6), (11, 6, 9)], 
[(8, 7, 2), (8, 2, 7), (2, 7, 8), (8, 6, 7), (6, 8, 7), (7, 6, 4), (6, 7, 8), (7, 8, 6), (2, 8, 7)], 
[(2, 8, 3), (8, 3, 2), (12, 11, 3), (3, 12, 11), (12, 3, 11), (11, 3, 12), (2, 3, 8), (11, 12, 10), (12, 11, 10)]]

# Calculate team-level diversity
res = list()
for l in best_teams:
    pairs = permutations(l, 2)
    res.append(mean([assess_hp_diversity(x[0], x[1]) for x in pairs]))

print("Mean team-level diversity: " + str(round(mean(res), 3)))
print("Range of team-level diversity: " + str(round(min(res), 3)) + " - " + str(round(max(res), 3)))

Mean team-level diversity: 0.829
Range of team-level diversity: 0.769 - 0.889
